In [1]:
#coding=utf-8
import numpy as np
import pandas as pd

In [2]:
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\jzqb_earlier6month.csv').decode('utf-8')
data_early = pd.read_csv(path, names=['shopid','list_user'])
data_early.head()

,shopid,list_user
0,1427469939,15026|1703212|5176157|5787401|6222925|6241420|...
1,1427495767,591220876|1101789123|1486286273|1544889055|172...
2,1427538754,1410509|2285814|2783100|14590476|15602920|2151...
3,1427550629,2129564|35210648|253893716|733200958|745665947...
4,1427582876,1096213|1410509|1478861|2285814|4469109|679745...


In [6]:
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\jzqb_later_weeks.csv').decode('utf-8')
data_later = pd.read_csv(path)
data_later['yearweek'] = data_later['year']*100 + data_later['week']
data_later.head()

,店铺id,year,week,销量,yearweek
0,1427469939,2015,43,7,201543
1,1427469939,2015,44,135,201544
2,1427469939,2015,45,102,201545
3,1427469939,2015,46,125,201546
4,1427469939,2015,47,101,201547


In [4]:
# 生成list_shop
list_shop = list(set(data_early['shopid'].values))
# 生成dict{shopid: user_sacle}
dict_shop_scale = {}
for shop in list_shop:
    scale = len(set(data_early[data_early['shopid']==shop]['list_user'].values[0].split('|')))
    dict_shop_scale[shop] = scale

In [5]:
# 生成了两两竞争强度（用户重合度），并保存
# path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\coincide.csv').decode('utf-8')
# f = open(path, 'w')
# f.write('shop1,shop2,coincide\r\n')
# for shop1 in list_shop:
#     for shop2 in list_shop:
#         if shop1 == shop2: continue
#         user1 = set(data_early[data_early['shopid']==shop1]['list_user'].values[0].split('|'))
#         user2 = set(data_early[data_early['shopid']==shop2]['list_user'].values[0].split('|'))
#         f.write(str(shop1) + ',' + str(shop2) + ',' + str(len(user1 & user2)) + '\r\n')
# f.close()
# 读取两两竞争强度
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\coincide.csv').decode('utf-8')
data_coincide = pd.read_csv(path)

In [33]:
# 每周市场大小
list_weeks = list(set(data_later['yearweek'].values))
temp = data_later.groupby('yearweek').sum()
dict_week_scale = temp.to_dict()
print dict_week_scale['销量']

{201600: 2712, 201601: 8140, 201602: 8871, 201603: 10086, 201604: 7999, 201605: 3705, 201606: 1222, 201607: 6693, 201608: 9691, 201609: 12549, 201610: 13928, 201611: 14520, 201612: 14250, 201613: 14156, 201614: 13970, 201615: 15430, 201616: 16411, 201617: 15078, 201618: 16372, 201619: 17211, 201620: 17819, 201621: 18436, 201622: 18257, 201623: 16588, 201624: 18254, 201625: 18817, 201626: 18815, 201627: 18719, 201628: 18931, 201629: 20372, 201630: 20820, 201631: 16721, 201632: 14897, 201633: 14992, 201634: 15130, 201635: 14831, 201636: 14015, 201637: 13463, 201638: 13641, 201639: 13049, 201640: 10681, 201641: 13167, 201642: 13661, 201643: 11868, 201644: 1475, 201543: 778, 201544: 5542, 201545: 5673, 201546: 6536, 201547: 6143, 201548: 6597, 201549: 7106, 201550: 7488, 201551: 7362, 201552: 4423}


In [34]:
# 将业绩表现写入data_later
## 1.市场份额比例（每月销量占市场总销量的比例）
def getPerformance_1(x):
    return float(x[2]) / dict_week_scale['销量'][x[1]]

data_later['performance_1'] = data_later[['店铺id','yearweek','销量']].apply(getPerformance_1,axis=1) 

In [56]:
# dict_shopid_shopname
import MySQLdb
# 连接数据库
conn=MySQLdb.connect(host='192.168.1.119',user='mysqlsmda',passwd='123456',db='smda',port=3306,charset='utf8')
sql = "SELECT shop_id, shop_name FROM smda.tb_baiduwaimaishop;"
data_shopidname = pd.read_sql(sql, conn)
keys = data_shopidname['shop_id'].values
values = data_shopidname['shop_name'].values
dict_shopid_shopname= dict(zip(keys, values))

In [35]:
data_later.head()

,店铺id,year,week,销量,yearweek,performance_1
0,1427469939,2015,43,7,201543,0.008997
1,1427469939,2015,44,135,201544,0.024359
2,1427469939,2015,45,102,201545,0.017980
3,1427469939,2015,46,125,201546,0.019125
4,1427469939,2015,47,101,201547,0.016441


## 1.面板数据实证

In [ ]:
# 定义绩效
dict_performance = {}
def getPerformance(shopid, time):
    return performanceShare(shopid, time) #市场份额比例（每月销量占市场总销量的比例）
#     return performanceShareIncreaseRate(shopid, time) ## 市场份额增长率((本月份额-上月份额)/上月份额)


## 市场份额比例（每月销量占市场总销量的比例）
def performanceShare(shopid, time):
    sale = data_later[(data_later['店铺id']==shopid) & (data_later['time']==time)]['销量'].values[0]
    return float(sale) / dict_month_scale['销量'][time]
## 市场份额增长率((本月份额-上月份额)/上月份额)
def performanceShareIncreaseRate(shopid, time):
    last_time = list_month[list_month.index(time)-1]
    sale_now = data_later[(data_later['店铺id']==shopid) & (data_later['time']==time)]['销量'].values[0]
    sale_now = float(sale_now) / dict_month_scale['销量'][time]
    sale_last = data_later[(data_later['店铺id']==shopid) & (data_later['time']==last_time)]['销量'].values[0]
    sale_last = float(sale_last) / dict_month_scale['销量'][last_time]
    return (sale_now-sale_last) / sale_last

In [ ]:
# 每家企业的top3竞争对手
dict_shop_competitor = {}
## 按照重合度(等同于 重合度/A企业用户规模)
for shop in list_shop:
    competitors = list(data_coincide[data_coincide['shop1']==shop].sort_values(by='coincide',ascending=False)['shop2'].values[:3])
    dict_shop_competitor[shop] = competitors

In [ ]:
# 筛选list_shop(面板数据)，考虑自身与竞争对手销量
list_shop_fliter = []
sale_limit = 50 ###每月销量阈值
def noSale(shopid):
    ### 后六个月必须每个月都有销量，且每月销量大于n
    sales = data_later[(data_later['店铺id']==shopid) & (data_later['time'].isin(list_month[-6:]))]['销量'].values
    if len(sales) < 6 or min(sales) < sale_limit:
        return True
    return False

for shop in list_shop:
    if noSale(shop): continue
    list_competitor = dict_shop_competitor[shop]
    if noSale(list_competitor[0]): continue
    if noSale(list_competitor[1]): continue
    if noSale(list_competitor[2]): continue
    list_shop_fliter.append(shop)

print '筛选后剩余' + str(len(list_shop_fliter)) + '家店铺'

In [ ]:
# 输出 月份，shopid，竞争对手1绩效，竞争对手2绩效，竞争对手3绩效
path = ('E:\\work\\竞争对手\\数据\\餐饮-计量\\result.csv').decode('utf-8')
f = open(path, 'w')
f.write('time,shopid,performance,last,competitor1,competitor2,competitor3\r\n')
for shopid in list_shop_fliter:
    months = list_month[-6:]
    for index,time in enumerate(months):
        list_competitor = dict_shop_competitor[shopid]
        print str(time) + ',' + str(shopid) + ',' + str(getPerformance(shopid, time))
        f.write(time + ',' + str(shopid) + ',' + str(getPerformance(shopid, time)) + ','\
                + str(getPerformance(shopid, months[index-1])) + ',' + str(getPerformance(list_competitor[0], time)) + ','\
                + str(getPerformance(list_competitor[1], time)) + ',' + str(getPerformance(list_competitor[2], time)) + '\r\n')
f.close()

In [ ]:
temp = pd.read_csv(path)
temp.head()

## 2.相关关系实证

In [39]:
# 筛选list_shop(相关关系实证)，只考虑自身销量
list_shop_fliter = []
sale_limit = 1 ###每周销量阈值
def noSale(shopid):
    ### 每周都需要有销量，且每月销量大于n
    sales = data_later[(data_later['店铺id']==shopid) & (data_later['yearweek'].isin(list_weeks[:]))]['销量'].values
    if len(sales) < 55 or min(sales) < sale_limit:
        return True
    return False

for shop in list_shop:
    if noSale(shop): continue
    list_shop_fliter.append(shop)
    
print '筛选后剩余' + str(len(list_shop_fliter)) + '家店铺'

筛选后剩余30家店铺


In [36]:
len(list_weeks)

55

In [40]:
data_filter_coincide = data_coincide[data_coincide['shop1'].isin(list_shop_fliter) & data_coincide['shop2'].isin(list_shop_fliter)].copy()

In [41]:
# 计算相对竞争强度（有向）
def getRelativeCoincideDirect(x):
    return float(x[1]) / dict_shop_scale[x[0]]
data_filter_coincide['coincide_relative_direct'] = data_filter_coincide[['shop2','coincide']].apply(getRelativeCoincideDirect, axis=1)

In [42]:
# 计算相对竞争强度（无向）
def getRelativeCoincideUnDirect(x):
    return float(x[2]) / (dict_shop_scale[x[0]] + dict_shop_scale[x[1]])
data_filter_coincide['coincide_relative_undirect'] = data_filter_coincide[['shop1','shop2','coincide']].apply(getRelativeCoincideUnDirect, axis=1)

In [43]:
data_filter_coincide.head()

,shop1,shop2,coincide,coincide_relative_direct,coincide_relative_undirect
15771,1486938201,1437948159,34,0.023843,0.013307
15772,1486938201,1453779204,1,0.003322,0.000699
15776,1486938201,1443502357,34,0.042607,0.017644
15788,1486938201,1434033553,185,0.089028,0.057686
15800,1486938201,1474003349,8,0.006563,0.003407


In [44]:
# 计算绩效相关性（shop1与shop2）
from scipy.stats import pearsonr
def getCorrelation(x):
    sales1 = data_later[(data_later['店铺id']==x[0]) & (data_later['yearweek'].isin(list_weeks[:]))]['performance_1'].values
    sales2 = data_later[(data_later['店铺id']==x[1]) & (data_later['yearweek'].isin(list_weeks[:]))]['performance_1'].values
    return pearsonr(sales1, sales2)[0]
def getCorrelation_p(x):
    sales1 = data_later[(data_later['店铺id']==x[0]) & (data_later['yearweek'].isin(list_weeks[:]))]['performance_1'].values
    sales2 = data_later[(data_later['店铺id']==x[1]) & (data_later['yearweek'].isin(list_weeks[:]))]['performance_1'].values
    return pearsonr(sales1, sales2)[1]

data_filter_coincide['correlation'] = data_filter_coincide[['shop1','shop2']].apply(getCorrelation, axis=1)
data_filter_coincide['p_value'] = data_filter_coincide[['shop1','shop2']].apply(getCorrelation_p, axis=1)

In [57]:
# 写入shop1和shop2的店名
def getShopName(x):
    if dict_shopid_shopname.has_key(str(x)):
        return dict_shopid_shopname[str(x)]
    return '未知'
data_filter_coincide['shop1_name'] = data_filter_coincide['shop1'].apply(getShopName)
data_filter_coincide['shop2_name'] = data_filter_coincide['shop2'].apply(getShopName)

In [62]:
data_filter_coincide.sort_values(by='correlation', ascending=False)[90:100]

,shop1,shop2,coincide,coincide_relative_direct,coincide_relative_undirect,correlation,p_value,shop1_name,shop2_name
54658,1474005418,1444301759,42,0.021898,0.015157,0.822191,1.407860e-14,食其家（裕德路店）,掌上韩品（万体星游城店）
135859,1444301759,1474005418,42,0.049238,0.015157,0.822191,1.407860e-14,掌上韩品（万体星游城店）,食其家（裕德路店）
39241,1443502357,1497393079,22,0.026699,0.013564,0.820351,1.804461e-14,蜀味鲜麻辣香锅（体育场）,澳门粥面莊（光启城）
299082,1497393079,1443502357,22,0.027569,0.013564,0.820351,1.804461e-14,澳门粥面莊（光启城）,蜀味鲜麻辣香锅（体育场）
235701,1438410296,1434033553,7,0.003369,0.002127,0.813680,4.336141e-14,蒋记麻辣烫(万航渡路),查厘士（徐汇店）
45860,1434033553,1438410296,7,0.005771,0.002127,0.813680,4.336141e-14,查厘士（徐汇店）,蒋记麻辣烫(万航渡路)
88158,1460377371,1434033553,35,0.016843,0.009063,0.812116,5.298445e-14,老盛兴汤包馆（定西一店）,查厘士（徐汇店）
45597,1434033553,1460377371,35,0.019619,0.009063,0.812116,5.298445e-14,查厘士（徐汇店）,老盛兴汤包馆（定西一店）
235907,1438410296,1440633944,4,0.004582,0.001918,0.809283,7.582393e-14,蒋记麻辣烫(万航渡路),丸龟制面(徐汇永新坊店)
161426,1440633944,1438410296,4,0.003298,0.001918,0.809283,7.582393e-14,丸龟制面(徐汇永新坊店),蒋记麻辣烫(万航渡路)


In [48]:
def getCorrelation_test(shop1, shop2):
    sales1 = data_later[(data_later['店铺id']==shop1) & (data_later['yearweek'].isin(list_weeks[:]))]['performance_1'].values
    sales2 = data_later[(data_later['店铺id']==shop2) & (data_later['yearweek'].isin(list_weeks[:]))]['performance_1'].values
#     print dict_shopid_shopname[shop1]
    print sales1
#     print dict_shopid_shopname[shop2]
    print sales2
    print pearsonr(sales1, sales2)

# getCorrelation_test(1443537695, 1450617003)

In [54]:
data_filter_coincide

,shop1,shop2,coincide,coincide_relative_direct,coincide_relative_undirect,correlation,p_value
15771,1486938201,1437948159,34,0.023843,0.013307,0.512664,6.289645e-05
15772,1486938201,1453779204,1,0.003322,0.000699,0.454904,4.843112e-04
15776,1486938201,1443502357,34,0.042607,0.017644,0.303074,2.449844e-02
15788,1486938201,1434033553,185,0.089028,0.057686,0.661960,3.727537e-08
15800,1486938201,1474003349,8,0.006563,0.003407,0.684899,8.067281e-09
15804,1486938201,1474005418,35,0.041032,0.017659,0.630151,2.533455e-07
15834,1486938201,1446849043,10,0.047393,0.007463,0.492193,1.352113e-04
15864,1486938201,1460377371,39,0.021861,0.013388,0.699185,2.894522e-09
15867,1486938201,1430645431,36,0.062827,0.021152,0.507749,7.592201e-05
15878,1486938201,1434223343,0,0.000000,0.000000,0.374225,4.883123e-03


In [53]:
# 绘图
import matplotlib.pyplot as plt
temp = data_filter_coincide[(data_filter_coincide['coincide_relative_undirect']>=0.00) & (data_filter_coincide['p_value']<=1.00)]
plt.scatter(temp['coincide_relative_undirect'],temp['correlation'])
plt.show()

## 3.网络生成

In [ ]:
# 点、线的阈值设置
threshold_node = 50
threshold_line = 10
threshold_line_per = 0.15

In [ ]:
# 生成dict_shopname_weight
dict_shopname_weight = {}
count = 0
for key in dict_shop_scale.keys():
    if dict_shopid_shopname.has_key(str(key)):
        if dict_shop_scale[key] >= threshold_node:
            shopname = dict_shopid_shopname[str(key)]
            dict_shopname_weight[shopname] = dict_shop_scale[key]
    else:
        count += 1
print '有' + str(count) + '个店家没有shopname'
print '剩余' + str(len(dict_shopname_weight.keys())) + '个店家'

In [ ]:
# 生成dict_shopnameline_weight
dict_shopnameline_weight = {}
dict_coincide = data_coincide[data_coincide['coincide']>threshold_line].to_dict('index') #线的绝对阈值
for i in dict_coincide.keys():
    if dict_shopid_shopname.has_key(str(dict_coincide[i]['shop1'])) and dict_shopid_shopname.has_key(str(dict_coincide[i]['shop2'])):
        line = dict_shopid_shopname[str(dict_coincide[i]['shop1'])] + ',' + dict_shopid_shopname[str(dict_coincide[i]['shop2'])]
        weight = float(dict_coincide[i]['coincide']) / dict_shop_scale[dict_coincide[i]['shop2']]
        if weight < threshold_line_per: continue #线的相对阈值
        dict_shopnameline_weight[line] = weight

In [ ]:
# 网络生成
import iNeedNet
path_ucinet = 'E:\work\竞争对手\数据\餐饮-网络\competitor_food.net'
path_gephi_node = 'E:\work\竞争对手\数据\餐饮-网络\competitor_food_node.csv'
path_gephi_line = 'E:\work\竞争对手\数据\餐饮-网络\competitor_food_line.csv'
iNeedNet.iNeedNet(dict_shopname_weight, dict_shopnameline_weight, True, False, path_ucinet, path_gephi_node, path_gephi_line)